In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f9d4cf2d9801a70dee6b232a92f8afc3b749ceae71f490dccd1d8b688bad44bd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np

import os

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# verify GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Convert to DataFrame for EDA

In [ ]:
def convert_data(filepath):

  # Read the data from the text file
  with open(filepath, "r") as file:
      lines = file.readlines()

  # Define an empty list to store the data
  data = []

  # Iterate over each line in the file
  for line in lines:
      # Split the line by spaces
      parts = line.strip().split()

      # Check if the line has the expected number of elements
      if len(parts) == 9:
          # Extract the values from the line
          text_file_name = parts[0]
          sentence_line_number = int(parts[1])
          sentence_word_index = int(parts[2])
          sentence_seq = parts[3]
          start_token = int(parts[4])
          end_token = int(parts[5])
          original_word = parts[6]
          word = parts[7]
          label = parts[8]

          # Append the values as a tuple to the data list
          data.append((text_file_name, sentence_line_number, sentence_word_index, sentence_seq,
                      start_token, end_token, original_word, word, label))

  # Create a DataFrame from the data list with appropriate column names
  df = pd.DataFrame(data, columns=['text_file_name', 'sentence_line_number', 'sentence_word_index',
                                  'sentence_seq', 'start_token', 'end_token', 'original_word',
                                  'word', 'label'])

  return df


In [ ]:
train_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_train.txt"
test_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_test.txt"

train = convert_data(train_data_path)
test = convert_data(test_data_path)

print(f"Length of train: {len(train)}")
print(f"Length of test: {len(test)}")


Length of train: 895141
Length of test: 585761


In [ ]:
df = train.copy()
df['sentence_line_number'].nunique()

1053

In [ ]:
df['label'].value_counts().sort_values()

label
I-Route           397
B-Duration        592
I-ADE             776
B-ADE             956
I-Duration       1034
I-Reason         3125
B-Reason         3791
I-Form           4173
B-Dosage         4221
I-Drug           4298
B-Route          5475
B-Frequency      6279
I-Strength       6617
B-Form           6647
B-Strength       6691
I-Dosage         8779
I-Frequency     13023
B-Drug          16222
O              802045
Name: count, dtype: int64

### Label labels

In [ ]:
labels = [i.split() for i in df['label'].values.tolist()]
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

print(unique_labels)

{'I-Route', 'I-Duration', 'I-Dosage', 'B-Dosage', 'B-Strength', 'B-Frequency', 'I-ADE', 'I-Strength', 'B-Form', 'I-Reason', 'B-Drug', 'I-Drug', 'I-Form', 'B-Duration', 'B-Route', 'I-Frequency', 'B-ADE', 'O', 'B-Reason'}


In [ ]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'B-ADE': 0, 'B-Dosage': 1, 'B-Drug': 2, 'B-Duration': 3, 'B-Form': 4, 'B-Frequency': 5, 'B-Reason': 6, 'B-Route': 7, 'B-Strength': 8, 'I-ADE': 9, 'I-Dosage': 10, 'I-Drug': 11, 'I-Duration': 12, 'I-Form': 13, 'I-Frequency': 14, 'I-Reason': 15, 'I-Route': 16, 'I-Strength': 17, 'O': 18}


In [ ]:
df[df['label'] == 'B-ADE']

,text_file_name,sentence_line_number,sentence_word_index,sentence_seq,start_token,end_token,original_word,word,label
637,data/training_20180910/110727.txt,68,3,T208,3011,3025,polyneuropathy,polyneuropathy,B-ADE
645,data/training_20180910/110727.txt,69,0,T377,3062,3070,infusion,infusion,B-ADE
700,data/training_20180910/110727.txt,74,1,T209,3237,3240,HTN,HTN,B-ADE
706,data/training_20180910/110727.txt,75,1,T210,3262,3265,DM2,DM0,B-ADE
732,data/training_20180910/110727.txt,78,1,T378,3365,3370,NAFLD,NAFLD,B-ADE
...,...,...,...,...,...,...,...,...,...
892895,data/training_20180910/118564.txt,156,6,T40,8092,8106,hallucinations,hallucinations,B-ADE
892897,data/training_20180910/118564.txt,157,1,T41,8111,8120,tachypnea,tachypnea,B-ADE
893013,data/training_20180910/118564.txt,168,10,T44,8731,8735,rash,rash,B-ADE
894424,data/training_20180910/100883.txt,128,0,T41,5871,5882,Hypotension,Hypotension,B-ADE


In [ ]:
def formatted_df(df):
  df['sentence'] = df[[
      'text_file_name',
      'sentence_line_number',
      'original_word',
      'label']].groupby(
          ['text_file_name', 'sentence_line_number'])['original_word'].transform(lambda x: ' '.join(x))

  df['word_labels'] = df[[
      'text_file_name',
      'sentence_line_number',
      'original_word',
      'label']].groupby(
          ['text_file_name', 'sentence_line_number'])['label'].transform(lambda x: ','.join(x))

  return df

In [ ]:
df = formatted_df(df)
df_test = formatted_df(test)

In [ ]:
def sentence_level_data_fn(df):
  sentence_level_data = df[["text_file_name", "sentence_line_number", "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
  return sentence_level_data

sentence_level_train = sentence_level_data_fn(df)
sentence_level_test = sentence_level_data_fn(df_test)

In [ ]:
sentence_level_data = sentence_level_train[["text_file_name", "sentence_line_number", "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
sentence_level_test = sentence_level_test[["text_file_name","sentence_line_number",  "sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)

sentence_level_data.head()

,text_file_name,sentence_line_number,sentence,word_labels
0,data/training_20180910/110727.txt,1,Admission Date : [ * * 2202 - 1 - 8 * * ] Disc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,data/training_20180910/110727.txt,3,Date of Birth : [ * * 2163 - 9 - 18 * * ] Sex : M,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,data/training_20180910/110727.txt,5,Service : MEDICINE,"O,O,O"
3,data/training_20180910/110727.txt,7,Allergies :,"O,O"
4,data/training_20180910/110727.txt,8,Keflex / Orencia / Remicade,"B-Drug,O,B-Drug,O,B-Drug"


In [ ]:
len(sentence_level_data)

83321

In [ ]:
sentence_level_data.iloc[7020].sentence

'your chemistries .'

In [ ]:
sentence_level_data.iloc[7020].word_labels

'O,O,O'

In [ ]:
print(sentence_level_data.iloc[21].sentence)
print(sentence_level_data.iloc[21].word_labels)

fx on x - ray . This was thought to be a psoriatic arthritis flare ,
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


#### Some sentences are really short. *Try* concatenating sentence to get longer inputs

In [ ]:
df = sentence_level_data.sort_values(by=['text_file_name', 'sentence_line_number'])
df_test = sentence_level_test.sort_values(by=['text_file_name', 'sentence_line_number'])

def cs(dataframe):
    processed_data_list = []

    # Group by 'text_file_name'
    grouped = dataframe.groupby("text_file_name")

    for name, group in grouped:
        sentences = group['sentence'].tolist()
        # Labels are already strings, so we take them as is
        labels = group['word_labels'].tolist()

        for i in range(0, len(sentences), 5):
            end_index = min(i + 5, len(sentences))
            current_batch = sentences[i:end_index]
            current_labels = labels[i:end_index]

            concatenated_sentence = " ".join(current_batch)
            # Concatenate labels as they are, assuming they're correctly formatted strings
            concatenated_label = ",".join(current_labels)

            processed_data_list.append({
                "text_file_name": name,
                "sentences": concatenated_sentence,
                "labels": concatenated_label
            })

    processed_data = pd.DataFrame(processed_data_list)
    return processed_data


# Correcting the function to properly handle labels
concatenated_df = cs(df)
concatenated_test= cs(df_test)

concatenated_df

In [ ]:
# make sure test is formatted same way
concatenated_df.iloc[301].sentences

"could be due to patient's liver dysfunction/third spacing from CHF . If cholecystitis is of clinical concern , HIDA scan can be performed provided the total bilirubin is not elevated . 3 ) Hyperdense renal cortex in left lower quadrant transplanted kidney . Findings are most likely due to chronic rejection or"

In [ ]:
concatenated_df.iloc[301].labels

'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'

In [ ]:
def contains_ade(labels):
    return any('ADE' in label for label in labels.split(','))

ade_mask = concatenated_df['labels'].apply(contains_ade)

# Use the mask to filter the DataFrame
ade_sentences_df = concatenated_df[ade_mask]

# Display the filtered DataFrame
ade_sentences_df

In [ ]:
data = concatenated_df.copy()
data_test = concatenated_test.copy()

data.rename(columns={'concatenated_sentence': 'sentence', 'concatenated_labels': 'word_labels'}, inplace=True)
data_test.rename(columns={'concatenated_sentence': 'sentence', 'concatenated_labels': 'word_labels'}, inplace=True)

### training

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForTokenClassification.from_pretrained(model, num_labels=len(labels_to_ids))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
TEST_BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentences[index].strip().split()
        word_labels = self.data.labels[index].split(",")
        # sentence_id = self.data.iloc[index]['sentence_id']

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True,
                             padding='max_length',
                             is_split_into_words=True,
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        item['index'] = index

        return item

  def __len__(self):
        return self.len

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(data, test_size=0.2, shuffle=False)


In [ ]:
train_dataset.reset_index(drop=True, inplace=True)
val_dataset.reset_index(drop=True, inplace=True)

In [ ]:
train_dataset['sent_id']  = train_dataset.index
val_dataset['sent_id']= val_dataset.index
data_test['sent_id'] = data_test.index


In [ ]:
print("FULL Dataet: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))
print("TEST Dataset: {}".format(data_test.shape))

data_test.reset_index(drop=True, inplace=True)

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
val_set = dataset(val_dataset, tokenizer, MAX_LEN)
test_set = dataset(data_test, tokenizer, MAX_LEN)

FULL Dataet: (16788, 3)
TRAIN Dataset: (13430, 4)
VAL Dataset: (3358, 4)
TEST Dataset: (11146, 4)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **test_params)


In [ ]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(3.0942, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 19])

In [ ]:
import torch.nn.functional as F


def focal_loss(logits, labels, alpha=0.25, gamma=2.0, ignore_index=-100):
    """
    logits: [batch_size, seq_len, num_labels] - model predictions
    labels: [batch_size, seq_len] - ground truth labels
    """
    # Calculate Cross Entropy Loss without reduction
    ce_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='none', ignore_index=ignore_index)

    # Get the predictions
    pred_probs = F.softmax(logits.view(-1, logits.size(-1)), dim=-1)
    pred_class = labels.view(-1)

    # focusing parameter
    gamma = gamma

    # Filter out 'ignore_index' labels
    filtered = labels.view(-1) != ignore_index

    # Calculate focal loss
    ce_loss_filtered = ce_loss[filtered]
    pred_probs_filtered = pred_probs[filtered]
    pred_class_filtered = pred_class[filtered]

    # Construct the loss
    pt = pred_probs_filtered.gather(1, pred_class_filtered.unsqueeze(-1)).squeeze()
    loss = ((1 - pt) ** gamma * ce_loss_filtered).mean()  # mean over the batch

    return loss



In [ ]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []

    model.train()

    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)
    #     global_attention_mask = torch.zeros(
    # input_ids.shape, dtype=torch.long, device=input_ids.device

        # Perform a forward pass to get the logits
        outputs = model(input_ids=ids, attention_mask=mask)

        # Calculate the loss using focal_loss function directly
        loss = focal_loss(outputs.logits, labels)  # Use the focal_loss function here directly

        tr_loss += loss.item()
        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # Decoding the logits to compute accuracy
        active_logits = outputs.logits.view(-1, model.num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)

        # Ignoring the predictions of the padding tokens
        active_accuracy = labels.view(-1) != -100
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(active_labels, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels.cpu().numpy())
        tr_preds.extend(predictions.cpu().numpy())

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")


In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.7527318000793457
Training loss per 100 training steps: 0.6569532255393149
Training loss per 100 training steps: 0.43135295575252963
Training loss per 100 training steps: 0.330070881404685
Training loss per 100 training steps: 0.271578968592578
Training loss per 100 training steps: 0.2299822101243093
Training loss per 100 training steps: 0.20182844447208328
Training loss per 100 training steps: 0.18308067930784339
Training loss per 100 training steps: 0.167499995832697
Training loss per 100 training steps: 0.15554645934580538
Training loss per 100 training steps: 0.14545559111132758
Training loss per 100 training steps: 0.13607852908903703
Training loss per 100 training steps: 0.1284007770539981
Training loss per 100 training steps: 0.12198305449735403
Training loss per 100 training steps: 0.11626609483431143
Training loss per 100 training steps: 0.11085445224404425
Training loss per 100 training steps: 0.10636127406104198
Traini

## Validate

In [ ]:
def valid(model, testing_loader):
    model.eval()
    eval_loss, eval_accuracy = 0, 0

    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, indices = [], [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)
            # indices_list = batch[idx]

            outputs = model(input_ids=ids, attention_mask=mask)
            loss = focal_loss(outputs.logits, labels)  # Use the same focal_loss function

            eval_loss += loss.item()
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # Decoding the logits to compute accuracy
            active_logits = outputs.logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)

            # Ignoring the predictions of the padding tokens
            active_accuracy = labels.view(-1) != -100
            active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

            labels = torch.masked_select(active_labels, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels.cpu().numpy())
            eval_preds.extend(predictions.cpu().numpy())

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

            indices.extend(batch['index'].tolist())



    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return indices,labels, predictions



In [ ]:
indices, labels, predictions = valid(model, val_loader)

Validation loss per 100 evaluation steps: 0.0038527334108948708
Validation loss per 100 evaluation steps: 0.02208062936748401
Validation loss per 100 evaluation steps: 0.02353187271341429
Validation loss per 100 evaluation steps: 0.025959991755861698
Validation loss per 100 evaluation steps: 0.02749102781167335
Validation loss per 100 evaluation steps: 0.030323969670753157
Validation loss per 100 evaluation steps: 0.03049074993808512
Validation loss per 100 evaluation steps: 0.030840792006965812
Validation loss per 100 evaluation steps: 0.029994755551667134
Validation loss per 100 evaluation steps: 0.02976542290071897
Validation loss per 100 evaluation steps: 0.0301041994225818
Validation loss per 100 evaluation steps: 0.02926984025281906
Validation loss per 100 evaluation steps: 0.029813305514830125
Validation loss per 100 evaluation steps: 0.029407437919168
Validation loss per 100 evaluation steps: 0.030037557049211065
Validation loss per 100 evaluation steps: 0.030126538546770044
Va

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         ADE       0.45      0.39      0.42       165
      Dosage       0.82      0.90      0.86       782
        Drug       0.89      0.93      0.91      3216
    Duration       0.61      0.76      0.67       120
        Form       0.91      0.88      0.90      1269
   Frequency       0.79      0.83      0.81      1270
      Reason       0.54      0.68      0.60       698
       Route       0.90      0.93      0.92      1143
    Strength       0.93      0.93      0.93      1341

   micro avg       0.84      0.88      0.86     10004
   macro avg       0.76      0.80      0.78     10004
weighted avg       0.85      0.88      0.86     10004



In [ ]:
# indices_df = pd.DataFrame(indices)
# indices_df.to_csv('indices_bioclinicalbert_concat7.csv', index=False)

### Test Set

In [ ]:
data_test['label_count'] = data_test['labels'].str.split(',').apply(len)

In [ ]:
def split_predictions(test_predictions, label_count):
    split_predictions = []
    start_idx = 0
    for count in label_count:
        end_idx = start_idx + count
        split_predictions.append(test_predictions[start_idx:end_idx])
        start_idx = end_idx
    return split_predictions
broken_up_test_preds = split_predictions(test_predictions, data_test['label_count'])



In [ ]:
data_test.iloc[30]['sentences']

In [ ]:
data_test.iloc[30]['labels']

In [ ]:
data_test.iloc[30]['predictions']

In [ ]:
indices, test_labels, test_predictions = valid(model, test_loader)

Validation loss per 100 evaluation steps: 0.010349477641284466
Validation loss per 100 evaluation steps: 0.024467365332132222
Validation loss per 100 evaluation steps: 0.02616212235077919
Validation loss per 100 evaluation steps: 0.026307430305515116
Validation Loss: 0.026203872918242718
Validation Accuracy: 0.9816380511133918


In [ ]:
# indices

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([test_labels], [test_predictions]))

              precision    recall  f1-score   support

         ADE       0.53      0.42      0.47       634
      Dosage       0.87      0.94      0.90      2704
        Drug       0.91      0.94      0.93     10595
    Duration       0.62      0.73      0.67       385
        Form       0.94      0.91      0.92      4373
   Frequency       0.82      0.85      0.84      4149
      Reason       0.53      0.62      0.57      2561
       Route       0.92      0.95      0.94      3513
    Strength       0.94      0.95      0.95      4237

   micro avg       0.86      0.89      0.88     33151
   macro avg       0.79      0.81      0.80     33151
weighted avg       0.87      0.89      0.88     33151



In [ ]:
def add_predictions_to_df(df, all_predictions):
    # adding predictions back to data_test for analysis
    prediction_slices = []

    start_idx = 0

    for index, row in df.iterrows():
        label_count = row['label_count']

        end_idx = start_idx + label_count

        # Slice the predictions for the current row
        current_predictions = all_predictions[start_idx:end_idx]
        prediction_slices.append(current_predictions)
        start_idx = end_idx


    df['predictions'] = prediction_slices

evaluate_test = add_predictions_to_df(data_test, test_predictions)

In [ ]:
len(indices)

11146

### Save

In [ ]:
# import os

# directory = "/content/drive/MyDrive/266_final/clinical_bert_concat8_take4"

# # if not os.path.exists(directory):
# #     os.makedirs(directory)

# tokenizer.save_vocabulary(directory)
# # model.save_pretrained(directory)

In [ ]:
# model.save_pretrained(directory, safe_serialization=False)